<a href="https://colab.research.google.com/github/hinekobot/Tensorflow2.0_sample/blob/master/01_LinearRegressionSample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow2.0  
- "Scikit-learnとTensorflowによる実践機械学習"を参考
- Chap.9を対象
- Tensorflow2.0で実施した場合の変更点等を確認


1. インストール
1. 最初のグラフの作成とセッション内での実行
  - Tensorflow2.0では`tf.Session()`が消滅
  - 値や式を入力したらその場で実行される
  - `.eval()`が消滅
    - 値の取得は`.numpy()`で可能
  - `tf.global_variables+initializer()`消滅
  - `tf.InteractiveSession()`消滅
1. グラフの管理
  - `tf.get_default_graph()`消滅
  - グラフの管理方法はどうなるか不明
1. ノード値のライフサイクル
  - `tf.Session()`が消滅しているので、この項目がどうなっているか不明
1. Tensorflowによる線形回帰
  - `tf.matrix_inverse()` -> `tf.linalg.inv()`に変更
1. 勾配降下法の実装
  - `tf.random_uniform()` -> `tf.randor.uniform()`に変更
  - `tf.assign()`がeager modeでは使えないので素直にループを作成
  - `tf.gradients()`がeager modeでは使えないので、`tf.GradientTape()`を使用する
  - `tf.train`が消滅しているため解説書で使用している`GradientDescentOptimizer()`が使用できない

1. 訓練アルゴリズムへのデータの供給
  - `tf.placeholder()`消滅
  - （現在確認中）
1. モデルの保存と復元
  - `tf.train.Saver()`消滅
  - `tf.saved_model()`を使う？
1. TensorBoardを使ったグラフと訓練曲線の可視化
  - (確認中)
1. 名前スコープ
1. モジュール性
1. 変数の共有

In [1]:
!pip install tensorflow==2.0.0-alpha0

    100% |████████████████████████████████| 79.9MB 384kB/s 
    100% |████████████████████████████████| 3.0MB 10.8MB/s 
    100% |████████████████████████████████| 61kB 25.7MB/s 
    100% |████████████████████████████████| 419kB 22.5MB/s 
  Found existing installation: tensorflow 1.13.1
    Uninstalling tensorflow-1.13.1:
      Successfully uninstalled tensorflow-1.13.1


### Check Version

In [58]:
import tensorflow as tf
tf.version.VERSION

'2.0.0-alpha0'

### Check Eager Execution
- Tensorflow1.xと異なり、変数や式を定義した時点で計算される

In [57]:
import tensorflow as tf

x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y +2

print(x)
print(y)
print(f)

<tf.Variable 'x:0' shape=() dtype=int32, numpy=3>
<tf.Variable 'y:0' shape=() dtype=int32, numpy=4>
tf.Tensor(42, shape=(), dtype=int32)


## Linear Regression
### by Using Normal Equation

In [4]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
XT = tf.transpose(X)

theta = tf.matmul(tf.matmul(tf.linalg.inv(tf.matmul(XT, X)), XT), y)

I0402 01:16:53.843023 139621377148800 california_housing.py:114] Downloading Cal. housing from https://ndownloader.figshare.com/files/5976036 to /root/scikit_learn_data


### Linear Regression by Using Stochastic Descent
- 解説書だと`tf.assign()`を使っているが、Tensorflow2.0では削除されている
- 仕方ないのでループで計算させている

In [0]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
#scaler = StandardScaler()
scaler = MinMaxScaler()

scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)

In [0]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
#theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta')
theta = tf.Variable(tf.random.uniform([n + 1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error)

for epoch in range(n_epochs):
  if epoch % 100 == 0:
    print('Epoch', epoch, 'MSE=', mse.numpy())
    
  theta = theta - learning_rate * gradients
  y_pred = tf.matmul(X, theta,name='theta')
  error = y_pred - y
  mse = tf.reduce_mean(tf.square(error),name='mse')
  gradients = 2/m * tf.matmul(tf.transpose(X), error)

best_theta = theta.numpy()

Epoch 0 MSE= 4.6538177
Epoch 100 MSE= 1.6631339
Epoch 200 MSE= 1.4238535
Epoch 300 MSE= 1.3489833
Epoch 400 MSE= 1.289773
Epoch 500 MSE= 1.236963
Epoch 600 MSE= 1.1893682
Epoch 700 MSE= 1.1463536
Epoch 800 MSE= 1.1073896
Epoch 900 MSE= 1.0720185


### Linear Regression by Using Stochastic Descent: Auto differentiation
- 解説書だと`tf.gradients`を使用しているが`egar execution`では使えない
- `eager execution`では`tf.GradientTape()`を使用する
  - `tape.watch()`で変数を監視する
  - 監視しないとうまく動かない
- 微分する変数をリストで渡す
- 返り値(`gradients`)もリストになっている

In [0]:
n_epochs = 1000
learning_rate = 0.001

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random.uniform([n + 1, 1], -1.0, 1.0), name='theta')

for epoch in range(n_epochs):
  if epoch % 100 == 0:
    print('Epoch', epoch, 'MSE=', mse.numpy())
    
  with tf.GradientTape() as g:
    g.watch(theta)
    y_pred = tf.matmul(X, theta,name='predictions')
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error),name='mse')

  gradients = g.gradient(mse, [theta])
  theta = theta - 0.01 * gradients[0]

best_theta = theta.numpy()

Epoch 0 MSE= 0.9298671
Epoch 100 MSE= 1.7388384
Epoch 200 MSE= 1.431847
Epoch 300 MSE= 1.363079
Epoch 400 MSE= 1.3122017
Epoch 500 MSE= 1.2661529
Epoch 600 MSE= 1.2237885
Epoch 700 MSE= 1.1847028
Epoch 800 MSE= 1.1485779
Epoch 900 MSE= 1.1151388


### Linear Regression by Using Stochastic Descent: Auto differentiation and optimizer
- optimizerを使って自動で変数(theta)が更新されるように
- 解説書だと`GradientDescentOptimizer`を使っているが、`tf.optimizer`には存在しないため、`RMSprop`を使用

In [0]:
n_epochs = 1000
learning_rate = 0.001

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random.uniform([n + 1, 1], -1.0, 1.0), name='theta')
optimizer = tf.optimizers.RMSprop(learning_rate=learning_rate)

for epoch in range(n_epochs):
  if epoch % 100 == 0:
    print('Epoch', epoch, 'MSE=', mse.numpy())
    
  with tf.GradientTape() as g:
    g.watch(theta)
    y_pred = tf.matmul(X, theta,name='predictions')
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error),name='mse')

  gradients = g.gradient(mse, [theta])
  optimizer.apply_gradients(zip(gradients, [theta]))

best_theta = theta.numpy()

Epoch 0 MSE= 2.1197662
Epoch 100 MSE= 4.097267
Epoch 200 MSE= 3.577104
Epoch 300 MSE= 3.113729
Epoch 400 MSE= 2.7070603
Epoch 500 MSE= 2.3569922
Epoch 600 MSE= 2.0633788
Epoch 700 MSE= 1.8260059
Epoch 800 MSE= 1.6445318
Epoch 900 MSE= 1.5183038


### Check: tf.function()

In [0]:
n_epochs = 1000
learning_rate = 0.001

@tf.function
def pred(X, theta):
  return tf.matmul(X, theta, name='predictions')

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random.uniform([n + 1, 1], -1.0, 1.0), name='theta')
optimizer = tf.optimizers.RMSprop(learning_rate=learning_rate)

for epoch in range(n_epochs):
  if epoch % 100 == 0:
    print('Epoch', epoch, 'MSE=', mse.numpy())
    
  with tf.GradientTape() as g:
    g.watch(theta)
    #y_pred = tf.matmul(X, theta,name='predictions')
    y_pred = pred(X, theta)
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error),name='mse')

  gradients = g.gradient(mse, [theta])
  optimizer.apply_gradients(zip(gradients, [theta]))

best_theta = theta.numpy()

Epoch 0 MSE= 1.4453444
Epoch 100 MSE= 5.184928
Epoch 200 MSE= 4.546678
Epoch 300 MSE= 3.9653354
Epoch 400 MSE= 3.440847
Epoch 500 MSE= 2.9731495
Epoch 600 MSE= 2.5621629
Epoch 700 MSE= 2.2077806
Epoch 800 MSE= 1.9098576
Epoch 900 MSE= 1.6681781


## TensorBoard

In [0]:
!mkdir ./tf_logs

In [14]:
from datetime import datetime as dt

now = dt.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = 'tf_logs'
logdir = '{}/run-{}'.format(root_logdir,now)


file_writer = tf.summary.create_file_writer(logdir)
#mse_summary = tf.summary.scalar('MSE', mse)

n_epochs = 1000
learning_rate = 0.001

@tf.function
def pred(X, theta):
  return tf.matmul(X, theta, name='predictions')

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random.uniform([n + 1, 1], -1.0, 1.0), name='theta')
optimizer = tf.optimizers.RMSprop(learning_rate=learning_rate)

for epoch in range(n_epochs):
  with tf.GradientTape() as g:
    g.watch(theta)
    #y_pred = tf.matmul(X, theta,name='predictions')
    y_pred = pred(X, theta)
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error),name='mse')

  gradients = g.gradient(mse, [theta])
  optimizer.apply_gradients(zip(gradients, [theta]))
  with file_writer.as_default():
    tf.summary.scalar('MSE', mse, step=epoch)
  
  if epoch % 100 == 0:
    print('Epoch', epoch, 'MSE=', mse.numpy())
    
best_theta = theta.numpy()


#file_writer.close()

Epoch 0 MSE= 7.4964085
Epoch 100 MSE= 6.5972657
Epoch 200 MSE= 5.833862
Epoch 300 MSE= 5.127448
Epoch 400 MSE= 4.4779935
Epoch 500 MSE= 3.8854563
Epoch 600 MSE= 3.349789
Epoch 700 MSE= 2.8709302
Epoch 800 MSE= 2.448808
Epoch 900 MSE= 2.083325


### Install LocalTunnel
- colabでTensorboardを実行するのに必要

In [9]:
!npm install -g localtunnel

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/client
+ localtunnel@1.9.1
added 55 packages from 34 contributors in 4.17s


### Call Tensorboard

In [54]:
print(logdir)
get_ipython().system_raw(
  'tensorboard --logdir {} --host 0.0.0.0 --port 6007 &'.format(logdir)
)
get_ipython().system_raw('lt --port 6007 >> url.txt 2>&1 &')

tf_logs/run-20190402012004


In [0]:
!cat url.txt

logを削除する用

In [0]:
!rm *.txt

In [0]:
!rm -rf ./tf_logs/*/*
!rmdir ./tf_logs/*

## Name Scope